# COVID learning gaps run deep among Los Angeles students, data shows 
By [Iris Lee](https://twitter.com/irisslee)  

Times examined report card grades received by students in six through 12th grades at the Los Angeles Unified School District. Share of Ds and Fs increased by more than five percentage points since before the pandemic. More than a quarter of grades in spring semester of 2020-2021 school year were Ds and Fs.

Read the full story [here](https://www.latimes.com/california/story/2021-10-21/covid-era-learning-challenges-lausd-after-school-closures)

In [1]:
import pandas as pd
import altair as alt

In [2]:
pd.set_option('display.max_columns', None)

## Import grades file for students in in grades six through 12

In [3]:
grades = pd.read_csv("input/all-grades-complete.csv")

Cleaning functions

In [4]:
cleanup = lambda x:x.replace(",","").replace("%","").replace("-","").replace(" ","") if type(x)==str else x

In [5]:
def simplify(x):
    season = x.split('SEMESTER')[0]
    year1 = x.split('SEMESTER')[1][:4]
    year2 = x.split('SEMESTER')[1][-4:]
    
    final = f"{year1}-{year2}-{season}"
    return final
    

Clean up

In [6]:
grades_clean = grades.applymap(cleanup)

In [7]:
grades_clean.columns = [x.replace("'","") for x in grades_clean.columns]

In [8]:
grades_clean["school_year"] = grades_clean["Semester School Year"].apply(simplify)

Change number columns to numeric

In [9]:
grades_clean = grades_clean.apply(pd.to_numeric, errors="ignore")

### Trim out Spring 2019-2020 but keep the rest  
No Fs were given in Spring 2019-2020 school year per LAUSD policy.

In [10]:
grades_trimmed = grades_clean[~grades_clean.school_year.str.contains("2019-2020-SPRING")].sort_values("school_year")

In [11]:
grades_trimmed

,Grades,Semester School Year,# of As,# of Bs,# of Cs,# of Ds,# of Fs,# of Ps,# of Ns,# of Is,Total Marks,% A,% B,% C,A-C %,% D,% F,% P,% N,% I,school_year
0,9to12,FALLSEMESTER20182019,301382,217981,189163,90078,101893.0,11342,21044,1308,934191,32.3,23.3,20.2,75.8,9.6,10.9,1.2,2.3,0.1,2018-2019-FALL
6,6to8,FALLSEMESTER20182019,221624,165766,130205,67168,54945.0,10174,66995,155,717032,30.9,23.1,18.2,72.2,9.4,7.7,1.4,9.3,0.0,2018-2019-FALL
1,9to12,SPRINGSEMESTER20182019,299486,210384,192475,88218,103546.0,15044,22173,1200,932526,32.1,22.6,20.6,75.3,9.5,11.1,1.6,2.4,0.1,2018-2019-SPRING
7,6to8,SPRINGSEMESTER20182019,214864,159205,131509,67244,65109.0,9029,67722,64,714746,30.1,22.3,18.4,70.7,9.4,9.1,1.3,9.5,0.0,2018-2019-SPRING
2,9to12,FALLSEMESTER20192020,306029,216670,186161,88136,95144.0,11645,19570,1149,924504,33.1,23.4,20.1,76.7,9.5,10.3,1.3,2.1,0.1,2019-2020-FALL
8,6to8,FALLSEMESTER20192020,221648,163995,126887,65126,51454.0,9242,58241,77,696670,31.8,23.5,18.2,73.6,9.3,7.4,1.3,8.4,0.0,2019-2020-FALL
4,9to12,FALLSEMESTER20202021,313109,157401,148168,121639,106001.0,127461,3566,372,977717,32.0,16.1,15.2,63.3,12.4,10.8,13.0,0.4,0.0,2020-2021-FALL
10,6to8,FALLSEMESTER20202021,204876,112867,109613,114456,48517.0,90312,2687,26,683354,30.0,16.5,16.0,62.5,16.7,7.1,13.2,0.4,0.0,2020-2021-FALL
5,9to12,SPRINGSEMESTER20202021,308459,149633,145530,113384,125127.0,123639,5255,1028,972055,31.7,15.4,15.0,62.1,11.7,12.9,12.7,0.5,0.1,2020-2021-SPRING
11,6to8,SPRINGSEMESTER20202021,204608,106101,107037,101245,71660.0,86882,2168,137,679838,30.1,15.6,15.7,61.4,14.9,10.5,12.8,0.3,0.0,2020-2021-SPRING


In [12]:
grades_trim = grades_trimmed[['Grades', 'Semester School Year', '# of As', '# of Bs', '# of Cs',
       '# of Ds', '# of Fs','Total Marks','school_year']].copy()

Groupby school year and sum

In [13]:
grades_combined = grades_trim.groupby("school_year").sum().reset_index()

In [14]:
grades_combined

,school_year,# of As,# of Bs,# of Cs,# of Ds,# of Fs,Total Marks
0,2018-2019-FALL,523006,383747,319368,157246,156838.0,1651223
1,2018-2019-SPRING,514350,369589,323984,155462,168655.0,1647272
2,2019-2020-FALL,527677,380665,313048,153262,146598.0,1621174
3,2020-2021-FALL,517985,270268,257781,236095,154518.0,1661071
4,2020-2021-SPRING,513067,255734,252567,214629,196787.0,1651893


Sum up letter grades

In [15]:
grades_combined["total_letter_grades"] = grades_combined[['# of As', '# of Bs', '# of Cs',
       '# of Ds', '# of Fs']].sum(axis=1)

Get share of the grades

In [16]:
def create_pct(df):
    cols = ["A","B","C","D","F"]
    for c in cols:
        df[f"{c}%"] = df[f"# of {c}s"]/df.total_letter_grades *100
        
    return df

In [17]:
grades_combined_pct = create_pct(grades_combined)

In [18]:
grades_combined_pct

,school_year,# of As,# of Bs,# of Cs,# of Ds,# of Fs,Total Marks,total_letter_grades,A%,B%,C%,D%,F%
0,2018-2019-FALL,523006,383747,319368,157246,156838.0,1651223,1540205.0,33.956908,24.915320,20.735422,10.209420,10.182930
1,2018-2019-SPRING,514350,369589,323984,155462,168655.0,1647272,1532040.0,33.572883,24.123978,21.147229,10.147385,11.008525
2,2019-2020-FALL,527677,380665,313048,153262,146598.0,1621174,1521250.0,34.687067,25.023172,20.578340,10.074741,9.636680
3,2020-2021-FALL,517985,270268,257781,236095,154518.0,1661071,1436647.0,36.055134,18.812415,17.943239,16.433752,10.755460
4,2020-2021-SPRING,513067,255734,252567,214629,196787.0,1651893,1432784.0,35.809096,17.848748,17.627709,14.979857,13.734589


## Findings

Pre-pandemic average share of Ds & Fs

In [19]:
prePandemic = grades_combined_pct[~grades_combined_pct.school_year.isin(["2020-2021-FALL","2020-2021-SPRING"])]

In [20]:
prePandemic[["# of Ds","# of Fs"]].values.sum()/prePandemic["Total Marks"].sum()

0.19067563285253541

Pandemic average share of Ds & Fs

In [21]:
pandemic = grades_combined_pct[grades_combined_pct.school_year.isin(["2020-2021-FALL","2020-2021-SPRING"])]

In [22]:
pandemic[["# of Ds","# of Fs"]].values.sum()/pandemic["Total Marks"].sum()

0.24208805166612132

Prep for chart

In [23]:
grades_chart = grades_combined_pct.melt(id_vars=["school_year"], value_vars=grades_combined_pct.columns[8:])

In [24]:
alt.Chart(grades_chart).mark_bar().encode(
    x='school_year',
    y='value',
    color='variable',  
).properties(
    title='Grades breakdown for Fall semesters',
    width=400,
    height=300
)

alt.Chart(...)

Export

In [25]:
grades_combined_pct.to_csv("output/combined.csv",index=False)

# Detailed break down of grades by ethnicy and student groups

In [27]:
hs_grades = pd.read_csv("input/grades9-12-complete.csv")

## High school

In [30]:
hs_grades_clean["school_year"] = hs_grades_clean["Semester School Year"].apply(simplify)

Clean up

In [29]:
hs_grades_clean.columns = [x.replace("'","") for x in hs_grades_clean.columns]

In [32]:
hs_no_col_list = hs_grades_clean.columns.to_list()

In [28]:
hs_grades_clean = hs_grades.applymap(cleanup)

In [ ]:
hs_no_col_list = [ s for s in hs_no_col_list if s not in ["type","groupname","Semester School Year","school_year"]]

Numeric columns

In [31]:
hs_grades_clean = hs_grades_clean.apply(pd.to_numeric, errors="ignore")

Trim to 2019 Fall and 2021 Spring

In [34]:
hs_1921 = hs_grades_clean[hs_grades_clean.school_year.isin(["2019-2020-FALL","2020-2021-SPRING"])].sort_values("school_year").copy()

## Middle School

In [37]:
middle = pd.read_csv("input/grades6-8-complete.csv")

Clean up

In [39]:
ms_grades_clean = middle.applymap(cleanup)

In [40]:
ms_grades_clean.columns = [x.replace("'","") for x in ms_grades_clean.columns]

In [41]:
ms_grades_clean["school_year"] = ms_grades_clean["Semester School Year"].apply(simplify)

Numeric columns

In [42]:
ms_grades_clean = ms_grades_clean.apply(pd.to_numeric, errors="ignore")

Trim to 2019 Fall and 2021 Spring

In [43]:
ms_1921 = ms_grades_clean[ms_grades_clean.school_year.isin(["2019-2020-FALL","2020-2021-SPRING"])].sort_values("school_year").copy()

## Combine all grades

In [44]:
combined_cols = ['type','groupname', 'Semester School Year', '# of As', '# of Bs',
       '# of Cs', '# of Ds', '# of Fs','school_year']

In [45]:
hs_ms_1921 = pd.concat([ms_1921[combined_cols],hs_1921[combined_cols]])

In [46]:
hs_ms_1921.type.value_counts()

ethnicity    36
group        12
Name: type, dtype: int64

In [47]:
hs_ms_combined= hs_ms_1921.groupby(["type","groupname","school_year"]).sum().reset_index()

In [48]:
hs_ms_combined["total_letter_grades"] = hs_ms_combined[['# of As', '# of Bs', '# of Cs',
       '# of Ds', '# of Fs']].sum(axis=1)

In [49]:
hs_ms_grouped_pct = create_pct(hs_ms_combined)

In [50]:
hs_ms_grouped_pct["A-C%"] = hs_ms_grouped_pct[['A%', 'B%', 'C%']].sum(axis=1)

In [51]:
hs_ms_grouped_pct["groupname"] = hs_ms_grouped_pct["groupname"].apply(lambda x: x.lower())

## Calculate changes

In [52]:
hs_ms_grouped_pct.reset_index(drop=True, inplace=True)

In [53]:
hs_ms_grouped_pct["A-C%_pt_change"] = hs_ms_grouped_pct.groupby(["groupname","type"])["A-C%"].diff()

Which group had large changes?

In [54]:
hs_ms_grouped_pct.sort_values("A-C%_pt_change").head(10)

,type,groupname,school_year,# of As,# of Bs,# of Cs,# of Ds,# of Fs,total_letter_grades,A%,B%,C%,D%,F%,A-C%,A-C%_pt_change
13,ethnicity,pacificislander,2020-2021-SPRING,565,248,253,267,224.0,1557.0,36.287733,15.928067,16.249197,17.148362,14.386641,68.464997,-15.038247
19,group,englishlearners,2020-2021-SPRING,39488,28398,36599,38100,36676.0,179261.0,22.028216,15.841706,20.416599,21.253926,20.459553,58.286521,-11.797839
9,ethnicity,hispanic,2020-2021-SPRING,346879,193100,200614,176182,166369.0,1083144.0,32.025197,17.827731,18.521452,16.265797,15.359823,68.374381,-10.205949
21,group,fostercareyouth,2020-2021-SPRING,3182,2343,3091,3382,3499.0,15497.0,20.533006,15.119055,19.945796,21.823579,22.578564,55.597858,-10.189321
23,group,socioeconomicallydisadvantaged,2020-2021-SPRING,414208,222253,230091,199825,184058.0,1250435.0,33.125112,17.774055,18.400876,15.980439,14.719518,69.300044,-9.774429
15,ethnicity,twoormore,2020-2021-SPRING,1585,768,653,561,473.0,4040.0,39.232673,19.009901,16.163366,13.886139,11.707921,74.405941,-9.108932
1,ethnicity,americanindian/alaskanative,2020-2021-SPRING,639,377,366,312,211.0,1905.0,33.543307,19.790026,19.212598,16.377953,11.076115,72.545932,-7.645273
11,ethnicity,other,2020-2021-SPRING,13180,4429,3441,2553,1977.0,25580.0,51.524629,17.314308,13.451916,9.980453,7.728694,82.290852,-6.589512
5,ethnicity,black,2020-2021-SPRING,32663,20428,22533,18524,14035.0,108183.0,30.192359,18.882819,20.828596,17.122838,12.973388,69.903774,-6.072148
17,ethnicity,white,2020-2021-SPRING,66049,22964,16389,11020,9435.0,125857.0,52.479401,18.246105,13.021922,8.755969,7.496603,83.747428,-5.006683


## Prep for chart

In [55]:
hs_ms_final = hs_ms_grouped_pct[['groupname','school_year','A-C%']].copy()

In [56]:
hs_ms_final["school_year"] = hs_ms_final["school_year"].apply(lambda x: x.split("-")[0])

In [57]:
hs_ms_final_chart = hs_ms_final.pivot(index='school_year',columns='groupname').reset_index().droplevel(0, axis=1)

In [58]:
hs_ms_final_chart

groupname,,americanindian/alaskanative,asian,black,englishlearners,filipino,fostercareyouth,hispanic,other,pacificislander,socioeconomicallydisadvantaged,twoormore,white
0,2019,80.191205,93.939765,75.975922,70.084360,92.010628,65.787178,78.580330,88.880364,83.503244,79.074473,83.514872,88.754111
1,2020,72.545932,89.604501,69.903774,58.286521,87.238679,55.597858,68.374381,82.290852,68.464997,69.300044,74.405941,83.747428


In [59]:
alt.Chart(hs_ms_final, title="2019/ 2020 comparison").mark_point().encode(
    alt.X(
        'A-C%:Q',
        title="A-C%",
        scale=alt.Scale(zero=False),
        axis=alt.Axis(grid=False)
    ),
    alt.Y(
        'groupname:N',
        title="",
        sort='x',
        axis=alt.Axis(grid=True)
    ),
    color=alt.Color('school_year:N', legend=alt.Legend(title="Year")),
).properties(
    height=alt.Step(20)
).configure_view(stroke="transparent")

alt.Chart(...)

In [60]:
hs_ms_final_chart.to_csv("output/combined-detail.csv",index=False)